# 메시지 필터링 (Filter Messages)

이 노트북에서는 **특정 조건에 맞는 메시지만 선택**하는 방법을 배웁니다.

## 필터링이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    필터링의 다양한 기준                            │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   필터링 기준:                                                     │
│                                                                    │
│   1️⃣ 유형(Type)으로 필터링                                        │
│      • human 메시지만 추출                                         │
│      • AI 메시지만 추출                                            │
│      • system 메시지 제외                                          │
│                                                                    │
│   2️⃣ 이름(Name)으로 필터링                                        │
│      • 특정 사용자의 메시지만                                      │
│      • 예시 메시지 제외                                            │
│                                                                    │
│   3️⃣ ID로 필터링                                                  │
│      • 특정 ID의 메시지 포함/제외                                  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 언제 필터링이 필요할까요?

| 상황 | 필터링 방법 |
|------|------------|
| Few-shot 예시 제거 | 예시 이름으로 제외 |
| 사용자 발언만 분석 | human 유형만 포함 |
| 특정 대화 제외 | ID로 제외 |

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 샘플 메시지 준비

다양한 속성을 가진 메시지들을 준비합니다.

In [ ]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

# 다양한 속성을 가진 메시지들
messages = [
    SystemMessage(content='당신은 친절한 어시스턴트입니다.', id='1'),
    HumanMessage(content='예시 입력', id='2', name='example_user'),      # 예시 메시지
    AIMessage(content='예시 출력', id='3', name='example_assistant'),    # 예시 메시지
    HumanMessage(content='실제 입력', id='4', name='bob'),               # 실제 사용자
    AIMessage(content='실제 출력', id='5', name='alice'),                # 실제 AI
]

print("=== 전체 메시지 ===")
for msg in messages:
    name = f" (name={msg.name})" if hasattr(msg, 'name') and msg.name else ""
    print(f"[{msg.id}] {msg.type}{name}: {msg.content}")

# 3. 유형(Type)으로 필터링

In [ ]:
from langchain_core.messages import filter_messages

# Human 메시지만 추출
human_messages = filter_messages(messages, include_types='human')

print("=== Human 메시지만 ===")
for msg in human_messages:
    print(f"[{msg.id}] {msg.content}")

In [ ]:
# AI 메시지만 추출
ai_messages = filter_messages(messages, include_types='ai')

print("=== AI 메시지만 ===")
for msg in ai_messages:
    print(f"[{msg.id}] {msg.content}")

In [ ]:
# 여러 유형 포함 (Human + AI)
human_ai_messages = filter_messages(messages, include_types=['human', 'ai'])

print("=== Human + AI 메시지 (System 제외) ===")
for msg in human_ai_messages:
    print(f"[{msg.id}] {msg.type}: {msg.content}")

# 4. 이름(Name)으로 필터링

In [ ]:
# 특정 이름 제외 (예시 메시지 제외)
excluded_examples = filter_messages(
    messages, 
    exclude_names=['example_user', 'example_assistant']
)

print("=== 예시 메시지 제외 ===")
for msg in excluded_examples:
    name = f" (name={msg.name})" if hasattr(msg, 'name') and msg.name else ""
    print(f"[{msg.id}] {msg.type}{name}: {msg.content}")

In [ ]:
# 특정 이름만 포함
bob_messages = filter_messages(
    messages,
    include_names=['bob']
)

print("=== bob의 메시지만 ===")
for msg in bob_messages:
    print(f"[{msg.id}] {msg.content}")

# 5. ID로 필터링

In [ ]:
# 특정 ID 제외
without_id3 = filter_messages(
    messages,
    exclude_ids=['3']  # ID가 '3'인 메시지 제외
)

print("=== ID '3' 제외 ===")
for msg in without_id3:
    print(f"[{msg.id}] {msg.type}: {msg.content}")

# 6. 복합 조건 필터링

In [ ]:
# 유형 + ID 조합 필터링
filtered = filter_messages(
    messages,
    include_types=['human', 'ai'],  # human과 ai만
    exclude_ids=['3']               # ID '3' 제외
)

print("=== human/ai 유형 중 ID '3' 제외 ===")
for msg in filtered:
    print(f"[{msg.id}] {msg.type}: {msg.content}")

# 7. 체인에서 필터 사용하기

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델
model = ChatOllama(model='llama3.2')

# 예시 메시지를 제외하는 필터
filter_ = filter_messages(exclude_names=['example_user', 'example_assistant'])

# 체인: 필터 → 모델
chain = filter_ | model

print("✅ 필터가 포함된 체인 생성 완료")

In [ ]:
# 체인 실행
# 전체 메시지를 전달하지만, 필터가 예시 메시지를 제외함
result = chain.invoke(messages)

print("=== 필터 체인 실행 결과 ===")
print(f"입력: 전체 {len(messages)}개 메시지")
print(f"필터 후: 예시 메시지 제외")
print(f"\nAI 응답: {result.content}")

---

## 정리: 메시지 필터링 (filter_messages)

### 주요 옵션

| 옵션 | 설명 | 예시 |
|------|------|------|
| **include_types** | 포함할 메시지 유형 | `'human'`, `['human', 'ai']` |
| **exclude_types** | 제외할 메시지 유형 | `'system'` |
| **include_names** | 포함할 이름 | `['bob']` |
| **exclude_names** | 제외할 이름 | `['example_user']` |
| **include_ids** | 포함할 ID | `['1', '2']` |
| **exclude_ids** | 제외할 ID | `['3']` |

### 핵심 코드

```python
from langchain_core.messages import filter_messages

# 1. 유형으로 필터링
human_msgs = filter_messages(messages, include_types='human')

# 2. 이름으로 필터링
filtered = filter_messages(messages, exclude_names=['example'])

# 3. 복합 조건
filtered = filter_messages(
    messages,
    include_types=['human', 'ai'],
    exclude_ids=['3']
)

# 4. 체인에서 사용
filter_ = filter_messages(exclude_names=[...])
chain = filter_ | model
```

### 활용 사례

| 사례 | 필터링 방법 |
|------|------------|
| Few-shot 예시 제거 | `exclude_names=['example_*']` |
| 사용자 발언만 분석 | `include_types='human'` |
| 시스템 메시지 제외 | `exclude_types='system'` |
| 특정 대화 제외 | `exclude_ids=['id']` |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**연속된 같은 유형의 메시지를 병합**하는 방법을 배웁니다. (15-16번 노트북)